In [1]:
import utils
import model
from torch.utils.data import DataLoader
from torch.optim import Adam

import torch
import os
import cv2

source_path = "./LargerChunks/hazy/"
target_path = "./Submission/trial1/"
paths = [source_path + x for x in os.listdir(source_path)]

/Users/lohith/miniconda3/envs/torch/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/lohith/miniconda3/envs/torch/lib/python3.8/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
TEST = utils.HeyZeeTest(paths)
TEST_LOADER = DataLoader(TEST, batch_size=1)

In [3]:
TheHayZee = model.MASK()

TheHayZee.load_state_dict(torch.load('./weights/model_11.pth', map_location=torch.device("cpu")))


<All keys matched successfully>

In [4]:
import numpy as np

def scale_to_norm(im):
    MIN = np.min(im.permute(1,2,0).numpy())
    MAX = np.max(im.permute(1,2,0).numpy())

    return np.array(255 * (im.permute(1,2,0).numpy() - MIN)/(MAX - MIN), dtype = np.int64)

In [5]:
TheHayZee.eval()  # Set the model to evaluation mode

with torch.no_grad():  # Disable gradient calculation
    for i, (path, haze) in enumerate(TEST_LOADER):
        I0, I1, I2, I3 = haze

        out = TheHayZee(I0, I1, I2, I3)

        to_write = target_path + path[0].split("/")[-1]

        saving_image = np.float32(scale_to_norm(I0[0]))

        print(np.min(saving_image), np.max(saving_image))

        cv2.imwrite(to_write, cv2.cvtColor(saving_image, cv2.COLOR_RGB2BGR))

: 